library

In [1]:
import pandas as pd , numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator

Load Data

In [205]:
data = pd.read_csv("D:/Project/Excel Datesheet/Uber.csv")
data.sample(n=5)

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
88,NaN,23,21,6,4,7,3
275,NaN,18,62,6,15,22,26
231,NaN,22,23,3,9,10,9
233,20-Sep-12,0,10,4,2,3,4
187,NaN,2,9,9,0,7,0


In [206]:
# missing value
data.isnull().sum()

Date                321
Time (Local)          0
Eyeballs              0
Zeroes                0
Completed Trips       0
Requests              0
Unique Drivers        0
dtype: int64

In [207]:
# duplicate values
data.duplicated().sum()

10

In [208]:
data[data.duplicated() == True] # They could be of different dates

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
69,NaN,4,1,1,0,0,0
118,NaN,5,1,1,0,1,0
163,NaN,2,3,3,0,1,0
189,NaN,4,1,1,0,0,0
213,NaN,4,1,1,0,0,0
236,NaN,3,2,2,0,2,0
238,NaN,5,1,1,0,0,0
262,NaN,5,1,1,0,0,0
332,NaN,3,3,3,0,1,0
333,NaN,4,1,1,0,0,0



### we see most values of Date column NULL. Since this is time series data and is ordered, we can forward fill the Date column to reflect
### different hours of same date

In [209]:
data['Date'].ffill(axis=0, inplace=True)
data.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_9160\974266189.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Date'].ffill(axis=0, inplace=True)


,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
0,10-Sep-12,7,5,0,2,2,9
1,10-Sep-12,8,6,0,2,2,14
2,10-Sep-12,9,8,3,0,0,14
3,10-Sep-12,10,9,2,0,1,14
4,10-Sep-12,11,11,1,4,4,11



Q1: Which date had the most completed trips during the two week period?

In [210]:
data.groupby('Date').agg(Total_Completed=('Completed Trips ', 'sum')).reset_index().sort_values(by='Total_Completed', ascending=False).head(1)

,Date,Total_Completed
12,22-Sep-12,248


Q2: What was the highest number of completed trips within a 24 hour period?

In [240]:
data['Datetime']=pd.to_datetime(data['Date'] + ' ' + data['Time (Local)'].astype(str)+':00:00')

C:\Users\hp\AppData\Local\Temp\ipykernel_9160\433646344.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Datetime']=pd.to_datetime(data['Date'] + ' ' + data['Time (Local)'].astype(str)+':00:00')


In [234]:
data.set_index('Datetime', inplace=True)
data_sum = data['Completed Trips '].rolling('24H').sum().shift(-23)

C:\Users\hp\AppData\Local\Temp\ipykernel_9160\1591637633.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data_sum = data['Completed Trips '].rolling('24H').sum().shift(-23)


In [235]:
data_sum.sort_values(ascending=False).head(1)

Datetime
2012-09-21 17:00:00    278.0
Name: Completed Trips , dtype: float64

278 Completed Trips between Sep 21, 2012 5 PM - Sep 22, 2012 5 PM

Q3: Which hour of the day had the most requests during the two week period?

In [236]:
data.groupby('Time (Local)').agg(Total_Requests=('Requests ' , 'sum')).reset_index().sort_values(by='Total_Requests', ascending=False).head(1)

,Time (Local),Total_Requests
23,23,184


Q4: What percentages of all zeroes during the two week period occurred on weekend (Friday at 5 pm to Sunday at 3 am)?

In [241]:
datetime = data['Datetime']=pd.to_datetime(data['Date'] + ' ' + data['Time (Local)'].astype(str)+':00:00')

C:\Users\hp\AppData\Local\Temp\ipykernel_9160\2974539667.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datetime = data['Datetime']=pd.to_datetime(data['Date'] + ' ' + data['Time (Local)'].astype(str)+':00:00')


In [253]:
# Extract the day of the week
#data = data.reset_index()

data['day_of_week'] = data['Datetime'].dt.day_name()
data['day']= data['Datetime'].dt.dayofweek
data.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Datetime,day_of_week,day
Datetime,,,,,,,,,,
2012-09-10 07:00:00,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,Monday,0
2012-09-10 08:00:00,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,Monday,0
2012-09-10 09:00:00,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,Monday,0
2012-09-10 10:00:00,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,Monday,0
2012-09-10 11:00:00,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,Monday,0


In [255]:
data['two_digit_of_hour'] = data['Time (Local)'].astype(str).str.zfill(2)
data.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Datetime,day_of_week,day,hour_two_digit,two_digit_of_hour
Datetime,,,,,,,,,,,,
2012-09-10 07:00:00,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,Monday,0,07,07
2012-09-10 08:00:00,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,Monday,0,08,08
2012-09-10 09:00:00,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,Monday,0,09,09
2012-09-10 10:00:00,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,Monday,0,10,10
2012-09-10 11:00:00,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,Monday,0,11,11


In [258]:
data[(data['day'].astype('str')+data['hour_two_digit']).astype('int').between(417,602)]['Zeroes '].sum()

641

In [261]:
data['Zeroes '].sum()

1429

In [275]:
round(data[(data['day'].astype('str')+data['hour_two_digit']).astype('int').between(417,602)]['Zeroes '].sum()/ data['Zeroes '].sum() * 100.0,2)

44.86

Q5: What is the weighted average ratio of completed trips per driver during the two week period?
+ Tip: "Weighted average" means your answer should account for the total trip volume in each hour to determine the most accurate number in whole period.

In [281]:
data['ratio']=data['Completed Trips ']/data['Unique Drivers']
data.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Datetime,day_of_week,day,hour_two_digit,two_digit_of_hour,ratio
Datetime,,,,,,,,,,,,,
2012-09-10 07:00:00,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,Monday,0,07,07,0.222222
2012-09-10 08:00:00,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,Monday,0,08,08,0.142857
2012-09-10 09:00:00,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,Monday,0,09,09,0.000000
2012-09-10 10:00:00,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,Monday,0,10,10,0.000000
2012-09-10 11:00:00,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,Monday,0,11,11,0.363636


In [284]:
data[data['ratio'].isnull()==True].head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Datetime,day_of_week,day,hour_two_digit,two_digit_of_hour,ratio
Datetime,,,,,,,,,,,,,
2012-09-11 05:00:00,11-Sep-12,5,1,1,0,0,0,2012-09-11 05:00:00,Tuesday,1,05,05,NaN
2012-09-12 02:00:00,12-Sep-12,2,3,3,0,1,0,2012-09-12 02:00:00,Wednesday,2,02,02,NaN
2012-09-12 03:00:00,12-Sep-12,3,2,2,0,0,0,2012-09-12 03:00:00,Wednesday,2,03,03,NaN
2012-09-12 04:00:00,12-Sep-12,4,1,1,0,0,0,2012-09-12 04:00:00,Wednesday,2,04,04,NaN
2012-09-13 00:00:00,13-Sep-12,0,11,11,0,2,0,2012-09-13 00:00:00,Thursday,3,00,00,NaN


In [285]:
# normal average ratio
np.average(data[data['Unique Drivers'] != 0]['ratio'])

0.4743397876937869

In [286]:
# weighted average ratio
num = sum(data[data['Unique Drivers'] != 0]['ratio'] * data[data['Unique Drivers'] != 0]['Completed Trips '])
denom = sum(data[data['Unique Drivers'] != 0]['Completed Trips '])
round(num/denom,2)

0.83

Q6. In drafting a driver schedule in terms of 8 hours shifts, when are the busiest 8 consecutive hours over the two week period in terms of unique requests?
+ A new shift starts in every 8 hours. Assume that a driver will work same shift each day.

In [292]:
data.set_index('Datetime', inplace=True)
df_rolling_sum = data['Requests '].rolling('8H').sum().shift(-7)

KeyError: "None of ['Datetime'] are in the columns"

In [293]:
df_rolling_sum.sort_values(ascending=False).head(5)

Datetime
2012-09-21 17:00:00    207.0
2012-09-22 19:00:00    207.0
2012-09-21 18:00:00    205.0
2012-09-22 18:00:00    202.0
2012-09-21 19:00:00    201.0
Name: Requests , dtype: float64

Busiest 8 consecutive hours with 207 unique requests each are:  
1. September 21, 2012 5 PM - September 22, 2012 1 AM  
2. September 22, 2012 7 PM - September 23, 2012 3 AM  

Q7: True or False: Driver supply always increases when demand increases during the two week period.
+ Tip: Visualize the data to confirm your answer if needed.

In [297]:
plt.figure(figsize=(15, 6))
plt.plot(data['Datetime'], data['Eyeballs '], color='red', label='Eyeballs')
plt.plot(data['Datetime'], data=['Unique Drivers'], color='blue', label='Drivers')
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.legend()
plt.xticks(rotation=90)
plt.show()

KeyError: 'Datetime'

<Figure size 1500x600 with 0 Axes>

https://github.com/everyday-data-science/Data_Science_Projects/blob/main/Insights%20from%20City%20Supply%20and%20Demand/Notebooks/Insights%20from%20City%20Supply%20and%20Demand.ipynb